In [1]:
import torch
import torchvision
from torchvision import transforms
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [3]:
#напишем нашу свёрточную нейронную сеть. 4 сверточных слоя 3x3 с maxpool и батч-норм.
#также три полносвязанных слоя, с дропаутом после каждого
#в целом, попытался регулиризовать нейронку с помощью аугментации, батч-норм и дропаута

class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()

        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.act1 = torch.nn.ReLU()
        self.bn1 = torch.nn.BatchNorm2d(num_features=16)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
       
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.act2 = torch.nn.ReLU()
        self.bn2 = torch.nn.BatchNorm2d(num_features=32)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.act3 = torch.nn.ReLU()
        self.bn3 = torch.nn.BatchNorm2d(num_features=64)
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.act4 = torch.nn.LeakyReLU()
        self.bn4 = torch.nn.BatchNorm2d(num_features=128)
        self.pool4 = torch.nn.MaxPool2d(kernel_size=3, stride=3)

        self.fc1 = torch.nn.Linear(128, 80)
        self.act5 = torch.nn.LeakyReLU()
        self.dropout1 = torch.nn.Dropout(0.5)

        self.fc2 = torch.nn.Linear(80, 80)
        self.act6 = torch.nn.LeakyReLU()
        self.dropout2 = torch.nn.Dropout(0.5)

        self.fc3 = torch.nn.Linear(80, 80)
        self.act7 = torch.nn.Tanh()
        self.dropout3 = torch.nn.Dropout(0.5)     

        self.fc4 = torch.nn.Linear(80, 10)
    
    def forward(self, x):

        x = self.conv1(x)
        x = self.act1(x)
        x = self.bn1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.act2(x)
        x = self.bn2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.act3(x)
        x = self.bn3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.act4(x)
        x = self.bn4(x)
        x = self.pool4(x)

        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        x = self.fc1(x)
        x = self.act5(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.act6(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.act7(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        
        return x

In [4]:
net = torch.load('/home/matvey/Morozov/Научрук/Диплом/Diploma-Thesis/Модель/Models/MNIST_net.pth', map_location=torch.device('cpu'))
net = net.to('cpu')
net.eval()

MyNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): LeakyReLU(negative_slope=0.01)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, trac

In [5]:
test_tf = transforms.Compose([
    transforms.ToTensor()
])

test = torchvision.datasets.MNIST('/home/matvey/Morozov/Научрук/Диплом/Diploma-Thesis/Модель/Datasets', download=False, train=False, transform=test_tf)
testloader = torch.utils.data.DataLoader(dataset=test, batch_size=256, shuffle=False)

In [6]:
labels = {
0 : 'T-shirt/top',
1 : 'Trouser',
2 : 'Pullover',
3 : 'Dress',
4 : 'Coat',
5 : 'Sandal',
6 : 'Shirt',
7 : 'Sneaker',
8 : 'Bag',
9 : 'Ankle boot'
}

In [7]:
from numpy.linalg import norm
from scipy.special import softmax

examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)

In [8]:
train = torchvision.datasets.MNIST('/home/matvey/Morozov/Научрук/Диплом/Diploma-Thesis/Модель/Datasets', download=False, train=True, transform=test_tf)
trainloader = torch.utils.data.DataLoader(dataset=test, batch_size=10000, shuffle=False)

In [9]:
train_examples = enumerate(trainloader)
batch_idx, (train_data, train_targets) = next(train_examples)

In [10]:
from boundary_MCMC import boundary_attack_MCMC
from boundary import boundary_attack

In [11]:
def nearest_image(image, label, train_data, train_targets):
    image = image.data.cpu().numpy()[0]
    all_norm = []
    for i, train_image in enumerate(train_data):
        if train_targets[i].data.cpu().numpy() != label:
            train_image = train_image.data.cpu().numpy()
            diff = train_image - image
            norm_diff = norm(diff)
            all_norm.append(norm_diff)
        else:
            all_norm.append(1e20)            

    index = np.argmin(all_norm)
    return index

#### Boundary attack

In [12]:
i = 0

l1_norm = []
l2_norm = []
probs_init = []
probs_adv = []

while True:
    class_label = example_targets[i].data.numpy()
    image = example_data[i]
    label = example_targets[i].data.numpy()

    prediction = net(torch.tensor(image.reshape(1, 1, 28, 28))).data.numpy()
    prob_init = np.max(softmax(prediction))

    index = nearest_image(image, label, train_data, train_targets)
    x_init = train_data[index].data.numpy().reshape(28,28)
    x_target = example_data[i].data.numpy().reshape(28,28)

    adv_example, proba = boundary_attack(net, x_init, x_target, threshold=None, verbose=1, max_iter=5e3)

    l1 = np.linalg.norm(adv_example - x_target, ord=1)
    l2 = np.linalg.norm(adv_example - x_target, ord=2)
    probs_init.append(prob_init)
    probs_adv.append(proba)
    l1_norm.append(l1)
    l2_norm.append(l2)
    
    i += 1
    
    if i >= 30:
        break

Start boundary attack
distance: 0.0000, itetarion: 5001
Start boundary attack
distance: 0.5619, itetarion: 5001
Start boundary attack
distance: 0.6263, itetarion: 5001
Start boundary attack
distance: 0.0000, itetarion: 5001
Start boundary attack
distance: 0.3193, itetarion: 5001
Start boundary attack
distance: 0.5181, itetarion: 5001
Start boundary attack
distance: 0.6271, itetarion: 5001
Start boundary attack
distance: 0.5064, itetarion: 5001
Start boundary attack
distance: 0.6245, itetarion: 5001
Start boundary attack
distance: 0.4559, itetarion: 5001
Start boundary attack
distance: 0.6131, itetarion: 5001
Start boundary attack
distance: 0.4260, itetarion: 5001
Start boundary attack
distance: 0.3828, itetarion: 5001
Start boundary attack
distance: 0.5558, itetarion: 5001
Start boundary attack
distance: 0.0000, itetarion: 5001
Start boundary attack
distance: 0.4061, itetarion: 5001
Start boundary attack
distance: 0.5479, itetarion: 5001
Start boundary attack
distance: 0.0000, itetario

In [13]:
print(l1_norm)
print(l2_norm)
print(probs_init)
print(probs_adv)

[8.530421231023187e-25, 2.0381643403446326, 1.964762841531189, 1.2066007734064625e-24, 1.0160556162820247, 1.825400314559647, 2.069057918945413, 1.7835189964439684, 2.315070623729047, 1.5414661131833163, 1.9856636945487698, 1.4965543131593129, 1.4624877410831185, 2.057409067882522, 7.969079549099725e-24, 1.3314506196938318, 2.1119457542188607, 4.625575031541875e-24, 0.11754995567576962, 1.696884014480543, 0.917818175641165, 1.88135969097396, 1.5868596985903127, 1.9615953637107904, 1.3397990021120032, 2.0831790635303156, 1.8947247126835718, 1.6551517516835874, 1.6332683814006301, 6.5501697368435195e-25]
[4.0144824409809235e-25, 0.5619416133992006, 0.6263076042201527, 9.714064478509077e-25, 0.3193172456190429, 0.5180831186287789, 0.6271141146536138, 0.5064044323973793, 0.6245324296144855, 0.45587039917925665, 0.6131055510003879, 0.42602888241196, 0.38282692083696257, 0.5557966510461799, 6.526212863116642e-24, 0.40613217616121516, 0.5479338690878782, 3.326644174773888e-24, 0.0353724242456

### Boundary MCMC

In [14]:
i = 0

l1_norm_MCMC = []
l2_norm_MCMC = []
probs_init_MCMC = []
probs_adv_MCMC = []

while True:
    class_label = example_targets[i].data.numpy()
    image = example_data[i]
    label = example_targets[i].data.numpy()
    
    prediction = net(torch.tensor(image.reshape(1, 1, 28, 28))).data.numpy()
    prob_init = np.max(softmax(prediction))
    
    index = nearest_image(image, label, train_data, train_targets)
    x_init = train_data[index].data.numpy().reshape(28,28)
    x_target = example_data[i].data.numpy().reshape(28,28)
    
    adv_example, proba = boundary_attack_MCMC(net, x_init, x_target, threshold=l2_norm[i], max_iter=2e4, verbose=1)
    
    l1 = np.linalg.norm(adv_example - x_target, ord=1)
    l2 = np.linalg.norm(adv_example - x_target, ord=2)
    probs_init_MCMC.append(prob_init)
    probs_adv_MCMC.append(proba)
    l1_norm_MCMC.append(l1)
    l2_norm_MCMC.append(l2)
    
    i += 1
    
    if i >= 30:
        break

Start boundary attack
distance: 0.0000, itetarion: 20001, alpha: 1.000, probability: 0.999
Start boundary attack
distance: 0.5629, itetarion: 11143, alpha: 0.455, probability: 0.409
Start boundary attack
distance: 0.6266, itetarion: 1333, alpha: 0.601, probability: 0.499
Start boundary attack
distance: 0.0000, itetarion: 1, alpha: 1.000, probability: 1.000
Start boundary attack
distance: 0.3192, itetarion: 9174, alpha: 0.512, probability: 0.504
Start boundary attack
distance: 0.5182, itetarion: 7723, alpha: 0.575, probability: 0.498
Start boundary attack
distance: 0.6289, itetarion: 8864, alpha: 0.703, probability: 0.469
Start boundary attack
distance: 0.5326, itetarion: 20001, alpha: 0.594, probability: 0.487
Start boundary attack
distance: 0.6244, itetarion: 19949, alpha: 0.392, probability: 0.207
Start boundary attack
distance: 0.4557, itetarion: 4800, alpha: 0.538, probability: 0.441
Start boundary attack
distance: 0.6137, itetarion: 18797, alpha: 0.606, probability: 0.282
Start bo

In [15]:
print(np.mean(probs_adv_MCMC))

0.5103644


In [16]:
print(np.mean(probs_adv))

0.51039475
